In [ ]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import random
import time
import numpy as np
import cv2
from PIL import Image, ImageEnhance

# --- Configuration ---
source_folder = r"/content/drive/MyDrive/Yellow_Mosaic_Virus"
target_folder = r"/content/drive/MyDrive/Yellow_Mosaic_Virus1"
target_images = 2000

# --- Ensure Target Folder Exists ---
os.makedirs(target_folder, exist_ok=True)

# --- PIL-Based Augmentation ---
def augment_image_pil(image, count):
    brightness_factor = random.uniform(0.7, 1.3)
    contrast_factor = random.uniform(0.7, 1.3)
    color_factor = random.uniform(0.7, 1.3)

    enhanced_image = ImageEnhance.Color(
        ImageEnhance.Contrast(
            ImageEnhance.Brightness(image).enhance(brightness_factor)
        ).enhance(contrast_factor)
    ).enhance(color_factor)

    filename = f"aug_pil_{count}_b{brightness_factor:.2f}_c{contrast_factor:.2f}_col{color_factor:.2f}.jpg"
    return enhanced_image, filename

# --- OpenCV-Based Augmentation ---
def augment_image_cv2(image_path, count):
    image = cv2.imread(image_path)
    if image is None:
        print(f"⚠️ Skipping invalid image: {image_path}")
        return []

    height, width = image.shape[:2]
    augmented_images = []

    # Center Crops
    for crop_ratio in [0.8, 0.6]:
        crop_h, crop_w = int(height * crop_ratio), int(width * crop_ratio)
        start_x, start_y = (width - crop_w) // 2, (height - crop_h) // 2
        cropped = image[start_y:start_y + crop_h, start_x:start_x + crop_w]
        resized = cv2.resize(cropped, (width, height), interpolation=cv2.INTER_AREA)
        filename = f"cv2_crop_{crop_ratio}_{count}.jpg"
        augmented_images.append((resized, filename))

    # Center Scaling
    for scale in [1.1, 1.3]:
        scaled_h, scaled_w = int(height * scale), int(width * scale)
        scaled = cv2.resize(image, (scaled_w, scaled_h), interpolation=cv2.INTER_LINEAR)
        crop_y = (scaled_h - height) // 2
        crop_x = (scaled_w - width) // 2
        cropped = scaled[crop_y:crop_y + height, crop_x:crop_x + width]
        filename = f"cv2_scale_{scale}_{count}.jpg"
        augmented_images.append((cropped, filename))

    # Flip
    augmented_images.append((cv2.flip(image, 1), f"cv2_flip_h_{count}.jpg"))
    augmented_images.append((cv2.flip(image, 0), f"cv2_flip_v_{count}.jpg"))

    # Rotation
    for angle in [15, -15, 30, -30]:
        M = cv2.getRotationMatrix2D((width // 2, height // 2), angle, 1)
        rotated = cv2.warpAffine(image, M, (width, height), flags=cv2.INTER_LINEAR)
        filename = f"cv2_rotate_{angle}_{count}.jpg"
        augmented_images.append((rotated, filename))

    # Shifting
    for dx, dy in [(10, 0), (-10, 0), (0, 10), (0, -10)]:
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        shifted = cv2.warpAffine(image, M, (width, height), flags=cv2.INTER_LINEAR)
        filename = f"cv2_shift_{dx}_{dy}_{count}.jpg"
        augmented_images.append((shifted, filename))

    return augmented_images

# --- Maintain Max Number of Images ---
def manage_target_folder(folder, max_images):
    images = sorted(
        [f for f in os.listdir(folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))],
        key=lambda x: os.path.getctime(os.path.join(folder, x))
    )
    if len(images) > max_images:
        for img in images[max_images:]:
            os.remove(os.path.join(folder, img))

# --- Main Processing ---
def main():
    if not os.path.isdir(source_folder):
        print(f"❌ Error: Source folder not found → {source_folder}")
        return

    image_paths = [
        os.path.join(source_folder, f)
        for f in os.listdir(source_folder)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]

    if not image_paths:
        print("❌ Error: No images found in the source folder.")
        return

    random.shuffle(image_paths)

    start_time = time.time()
    current_count = len(os.listdir(target_folder))

    while current_count < target_images:
        for img_path in image_paths:
            if current_count >= target_images:
                break
            try:
                with Image.open(img_path) as pil_image:
                    if pil_image.mode != "RGB":
                        pil_image = pil_image.convert("RGB")

                    # Convert PNG to JPG if needed
                    pil_aug, pil_filename = augment_image_pil(pil_image, current_count)
                    save_path = os.path.join(target_folder, pil_filename)
                    pil_aug.save(save_path, format="JPEG", quality=95)
                    print(f"✅ Saved PIL image: {pil_filename}")
                    current_count += 1

                # OpenCV augmentations
                cv2_augments = augment_image_cv2(img_path, current_count)
                for aug_img, filename in cv2_augments:
                    if current_count >= target_images:
                        break
                    cv2.imwrite(os.path.join(target_folder, filename), aug_img)
                    print(f"✅ Saved CV2 image: {filename}")
                    current_count += 1

            except Exception as e:
                print(f"⚠️ Error with {img_path}: {e}")

    manage_target_folder(target_folder, target_images)
    print(f"\n🎯 Augmentation complete. Total images: {len(os.listdir(target_folder))}")
    print(f"⏱ Time taken: {time.time() - start_time:.2f} seconds")

# --- Run Script ---
if __name__ == "__main__":
    main()


✅ Saved PIL image: aug_pil_0_b1.27_c1.19_col0.99.jpg
✅ Saved CV2 image: cv2_crop_0.8_1.jpg
✅ Saved CV2 image: cv2_crop_0.6_1.jpg
✅ Saved CV2 image: cv2_scale_1.1_1.jpg
✅ Saved CV2 image: cv2_scale_1.3_1.jpg
✅ Saved CV2 image: cv2_flip_h_1.jpg
✅ Saved CV2 image: cv2_flip_v_1.jpg
✅ Saved CV2 image: cv2_rotate_15_1.jpg
✅ Saved CV2 image: cv2_rotate_-15_1.jpg
✅ Saved CV2 image: cv2_rotate_30_1.jpg
✅ Saved CV2 image: cv2_rotate_-30_1.jpg
✅ Saved CV2 image: cv2_shift_10_0_1.jpg
✅ Saved CV2 image: cv2_shift_-10_0_1.jpg
✅ Saved CV2 image: cv2_shift_0_10_1.jpg
✅ Saved CV2 image: cv2_shift_0_-10_1.jpg
✅ Saved PIL image: aug_pil_15_b0.73_c0.80_col0.95.jpg
✅ Saved CV2 image: cv2_crop_0.8_16.jpg
✅ Saved CV2 image: cv2_crop_0.6_16.jpg
✅ Saved CV2 image: cv2_scale_1.1_16.jpg
✅ Saved CV2 image: cv2_scale_1.3_16.jpg
✅ Saved CV2 image: cv2_flip_h_16.jpg
✅ Saved CV2 image: cv2_flip_v_16.jpg
✅ Saved CV2 image: cv2_rotate_15_16.jpg
✅ Saved CV2 image: cv2_rotate_-15_16.jpg
✅ Saved CV2 image: cv2_rotate_30_1